# Assignment 2 - MATH60837A
William Bourque - 11359215

Frederic Pelletier - 11359258

## installing dependencies (only needed once)

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install pandas
%pip install statsmodels

## importing dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from statsmodels.tsa.stattools import acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import chi2
import warnings
warnings.filterwarnings("ignore")

## Loading the data

In [ ]:
df = pd.read_csv('./data.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)
df['log_HP'] = np.log(df['QCAR628BIS'])
df['diff_HP'] = df['log_HP'].diff()
df = df.dropna()
yt = df['diff_HP']
df.head()

# Part1.Model Selection

### 1.a) Estimate all 4 models by maximum likelihood, report the estimation results, and verify if the stationarity conditions and invertibility conditions are satisfied

In [49]:
def test_stationarity(model: ARIMA):
    p,_,q = model.specification.order
    r = max(p,q)
    phi_matrix = np.zeros((r,r))
    phis = model.arparams
    phi_matrix[0,:] = np.pad(model.arparams, (0, r - len(phis)), mode='constant')
    np.fill_diagonal(phi_matrix[1:],1)
    eigenvalues = np.linalg.eigvals(phi_matrix)
    print(phi_matrix)
    return np.all(np.abs(eigenvalues) < 1)

def test_invertibility(model:ARIMA):
    p,_,q = model.specification.order
    r = max(p,q)
    theta_matrix = np.zeros((r,r))
    thetas = model.maparams
    theta_matrix[0,:] = np.pad(thetas, (0,r - len(thetas)), mode='constant')
    np.fill_diagonal(theta_matrix[1:],1)
    eigenvalues = np.linalg.eigvals(theta_matrix)
    print(theta_matrix)
    return np.all(np.abs(eigenvalues) < 1)

In [ ]:
#since we will be testing for stationarity and invertibility, we do not force the
# algorithm to ensure either of them
ar4 = ARIMA(yt, order=(4,0,0), enforce_stationarity=False, enforce_invertibility=False).fit()
arma41 = ARIMA(yt, order=(4,0,1),enforce_stationarity=False, enforce_invertibility=False).fit()
arma42 = ARIMA(yt, order=(4,0,2),enforce_stationarity=False, enforce_invertibility=False).fit()


for name,model in {'ar(4)': ar4,'arma(4,1)': arma41,'arma(4,2)': arma42}.items():
    stationary = test_stationarity(model)
    invertible = test_invertibility(model)
    print(model.summary())
    print(f'{name} is {"stationary" if stationary else "not stationary"}')
    print(f'{name} is {"invertible" if invertible else "not invertible"}')
    

### Interpretation
All models are both stationary and invertible since the eigenvalues of their respective phi and theta matrices all have modules less than 1. However, for all models, some parameters 
do not have a p-value < 0.05, meaning there is no strong statistical evidence that the parameter should be different than 0

### 1.b) 